# Data Processing

## 0. Dependencies

In [1]:
# !pip install ipynb

import os
import cv2
import pandas as pd
from PIL import Image
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm


In [2]:
train_dir = '/opt/ml/input/data/train'
train_img_dir = os.path.join(train_dir, 'images')
train_img_sub_dirs = [os.path.join(train_img_dir, sub_dir) for sub_dir in os.listdir(train_img_dir) if os.path.isdir(os.path.join(train_img_dir, sub_dir))]

## 1. Dataset

In [3]:
default_img_paths = np.array([[os.path.join(sub_dir, img) for img in os.listdir(sub_dir) if not img.startswith('.')]  for sub_dir in train_img_sub_dirs]).flatten()

default_transforms = transforms.Compose([
    transforms.ToTensor()
])

In [8]:
class TrainDataset(Dataset):
    def __init__(self, img_paths=default_img_paths, transforms=default_transforms):
        self.train_info = pd.read_csv(os.path.join(train_dir, 'train.csv'))
        self.train_info['label_weight'] = self._cal_label_weight(self.train_info['gender'], self.train_info['age'])
        
        self.img_paths = img_paths
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_paths[index]
        image = Image.open(img_path)
        tmp_dir, img_name = os.path.split(img_path)
        _, person = os.path.split(tmp_dir)
        
        if self.transforms:
            image = self.transforms(image)
        
        y = self._toY(person, img_name)
        return image, y

    def __len__(self):
        return len(self.img_paths)
    
    def _cal_label_weight(self, gender, age):
        weight = np.zeros(gender.shape)
        # gender
        weight += (gender == 'female')*3
        # age
        weight += ((age >= 30) & (age < 60))*1
        weight += (age >= 60)*2

        return weight
    
    def _toY(self, person, mask_label):
        label_weight = self.train_info.query(f"path == '{person}'")['label_weight'].values[0]
        mask_weight = 0
        if mask_label.startswith('incorrect'):
            mask_weight += 6
        elif mask_label.startswith('normal'):
            mask_weight += 12
        return label_weight + mask_weight
        
    
    def __repr__(self):
        idx = np.random.randint(len(self))
        X, y = self[idx]
        return f'[{self.__class__.__name__}]\n\t length : {len(self)} \n\t y : {y} \n\t X.shape : {X.shape} \n\t X : \n{X}'

In [10]:

train_set = TrainDataset()

In [11]:
print(train_set)

[TrainDataset]
	 length : 18900 
	 y : 12.0 
	 X.shape : torch.Size([3, 512, 384]) 
	 X : 
tensor([[[0.7412, 0.7451, 0.7490,  ..., 0.6667, 0.6667, 0.6667],
         [0.7451, 0.7451, 0.7490,  ..., 0.6667, 0.6667, 0.6667],
         [0.7490, 0.7490, 0.7529,  ..., 0.6667, 0.6667, 0.6667],
         ...,
         [0.3216, 0.4000, 0.6157,  ..., 0.0118, 0.1294, 0.2824],
         [0.3569, 0.5412, 0.6902,  ..., 0.0157, 0.0863, 0.1961],
         [0.4078, 0.6392, 0.7098,  ..., 0.0196, 0.0549, 0.1373]],

        [[0.7294, 0.7333, 0.7373,  ..., 0.6549, 0.6549, 0.6549],
         [0.7333, 0.7333, 0.7373,  ..., 0.6549, 0.6549, 0.6549],
         [0.7373, 0.7373, 0.7412,  ..., 0.6549, 0.6549, 0.6549],
         ...,
         [0.3255, 0.4039, 0.6196,  ..., 0.0118, 0.1294, 0.2824],
         [0.3608, 0.5451, 0.6941,  ..., 0.0157, 0.0863, 0.1961],
         [0.4118, 0.6431, 0.7137,  ..., 0.0196, 0.0549, 0.1373]],

        [[0.7020, 0.7059, 0.7098,  ..., 0.6275, 0.6275, 0.6275],
         [0.7059, 0.7059, 0.7098

## 2. DataLoader

In [34]:
train_img_iter_basic = DataLoader(train_set)
train_img_iter_batch = DataLoader(train_set,
                           batch_size=100
                           )
train_img_iter_numworker = DataLoader(train_set,
                           num_workers=3
                           )
train_img_iter_numworker_batch = DataLoader(train_set,
                            batch_size=100,
                            num_workers=3
                           )

In [35]:
# for i, (data, target) in tqdm(enumerate(train_img_iter_basic)):
#     if i == 1000:
#         break
        

In [36]:
# for i, (data, target) in enumerate(tqdm(train_img_iter_batch)):
#     if i == 10:
#         break

In [37]:
# for i, (data, target) in enumerate(tqdm(train_img_iter_numworker)):
#     if i == 1000:
#         break

In [38]:
# for i, (data, target) in enumerate(tqdm(train_img_iter_numworker_batch)):
#     if i == 10:
#         break